In [41]:
import time
from importlib.resources import files
import numpy as np
from Consensus import *
import Consensus
from collections import OrderedDict
from bitstring import Bits

In [ ]:
INPUT_TREE_PATH = files("Consensus.sample10").joinpath(f"lowsample3.tre")
input_trees = TreeList_with_support.get(path = INPUT_TREE_PATH, schema = "newick")
majority = input_trees.majority_rule_consensus()

In [ ]:
stdg = STDGreedyConsensus(input_trees)

In [ ]:
stdg.BipartitionP

In [ ]:
stdg.DIST.shape

In [ ]:
stdg.n_bipartitions

In [ ]:
over_bipartitions = np.zeros(stdg.n_bipartitions)
for i in range(stdg.n_bipartitions):
    over_count = 0
    for j in range(stdg.n_bipartitions):
        if(stdg.DIST[i,j] < stdg.BipartitionP[i] - 1): over_count += 1
    over_bipartitions[i] = over_count/stdg.n_bipartitions

In [ ]:
over_bipartitions

In [ ]:
np.zeros(len(stdg.BipartitionList))

In [ ]:
len(stdg.BipartitionList)

In [11]:
ASTRAL_INPUT_TREE_PATH = files("Consensus.sample100").joinpath(f"lowsample3.tre")
ASTRAL_TREE_PATH = files("Consensus.sample100").joinpath(f"low_astral3.tre")

In [12]:
astral_input_trees=TreeList_with_support.get(path = ASTRAL_INPUT_TREE_PATH, schema = "newick")
astral_tree = Tree_with_support.get(path = ASTRAL_TREE_PATH, schema = "newick",taxon_namespace = astral_input_trees.taxon_namespace)

In [46]:
astral_std = astral_tree.STD_greedy_pruning(astral_input_trees, normalized=True,time_flag =True)

computing transfer_support...
computing transfer_support for the other side...
current risk: 68506.74717144936
current risk: 67619.63050478269
current risk: 66743.38764763984
current risk: 65871.2066952589
current risk: 65042.40669525889
current risk: 64213.60669525889
current risk: 63388.573361925555
current risk: 62572.45669525889
current risk: 61760.62336192555
current risk: 60971.74002859222
current risk: 60207.00669525889
current risk: 59445.850742877934
current risk: 58716.9174095446
current risk: 58079.250742877935
current risk: 57443.78407621127
current risk: 56809.95074287793
current risk: 56198.72574287793
current risk: 55612.36643551862
current risk: 55038.36643551862
current risk: 54486.36643551862
current risk: 53935.53310218528
current risk: 53396.934148361324
current risk: 52859.57343407561
current risk: 52327.5996245518
current risk: 51855.0996245518
current risk: 51407.79367217085
current risk: 50961.3997327769
current risk: 50520.37949468166
current risk: 50110.874335

In [ ]:
len(astral_std.bipartition_encoding)

In [17]:
len(astral_input_trees)

1000

In [26]:
#treelistのBipartitionListを作成する
BipartitionCountDict = OrderedDict()
BipartitionDict = OrderedDict()
BipartitionList = [];BipartitionBitsList = []
for tree in astral_input_trees:
    dict_keys = BipartitionCountDict.keys() # no duplicate keys in one tree
    tree.encode_bipartitions()
    for branch in tree.internal_edges(exclude_seed_edge=True):
        key = branch.bipartition.split_as_int()
        BipartitionDict.setdefault(key, branch.bipartition)
        if key in dict_keys:
            BipartitionCountDict[key] += 1
        else:
            BipartitionCountDict[key] = 1
for key, value in BipartitionCountDict.items():
    BipartitionList.append(key)
    bitstr = Bits(uint = key, length=astral_tree.n_taxa)
    BipartitionBitsList.append(bitstr)

In [30]:
Astral_BiparitionDict = OrderedDict()
Astral_BipartitionList = [];Astral_BipartitionBitsList = []

for branch in astral_tree.internal_edges(exclude_seed_edge=True):
    key = branch.bipartition.split_as_int()
    Astral_BiparitionDict.setdefault(key, branch.bipartition)
    Astral_BipartitionList .append(key)
    bitstr = Bits(uint = key, length=astral_tree.n_taxa)
    Astral_BipartitionBitsList.append(bitstr)

In [42]:
#Astral_tree と Consensus treeの全てのDIST計算をする
Astral_DIST = np.zeros((len(Astral_BipartitionList), len(BipartitionList)))
for i in range(len(Astral_BipartitionList)):
    for j in range(len(BipartitionList)):
        Astral_DIST[i,j] = Consensus._greedy._MinHammingDist(Astral_BipartitionBitsList[i],BipartitionBitsList[j])

In [44]:
#argsortする
ORDER = np.zeros((len(Astral_BipartitionList), len(BipartitionList)))
for i in range(len(Astral_BipartitionList)):
    ORDER[i] = np.argsort(Astral_DIST[i])

In [45]:
ORDER[3]

array([   11.,  5513., 10129., ...,  3633.,  9948.,   120.])

In [ ]:
#メモ
